In [13]:
import numpy as np
import random
from scipy.stats import norm
from utils.bs_funcs import *

# Define initial parameters
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to maturity in years
r = 0.05 # Risk-free interest rate
sigma = 0.2  # Volatility of the underlying stock

# Define the options structure

option = {
    'S': S,
    'K': K,
    'T': T,
    'r': r,
    'sigma': sigma,
    'type': random.choice(list(['c', 'p']))
}

# Function to randomly shock one of the variables
def shock_variable(option):
    variable_to_shock = random.choice(list(option.keys()))
    shock_factor = random.choice([0.9, 1.1])  # Random shock between -10% to +10%
    option[variable_to_shock] = option[variable_to_shock] * shock_factor
    print(f'option type: {option['type']}')
    return variable_to_shock, shock_factor, option

# Original option price
original_price = black_scholes(**option)

# Shock one variable
variable_to_shock, shock_factor, shocked_option = shock_variable(option)

# New option price after shock
new_price = black_scholes(**shocked_option)

print(f"Original Option Price: {original_price:.2f}")
print(f"Shocked Variable: {variable_to_shock}")
print(f"Shock Factor: {shock_factor:.2f}")


# Ask user for prediction
user_prediction = input("Predict the outcome on option pricing (higher/lower/same): ").strip().lower()

# Determine actual outcome
outcome = 'higher' if new_price > original_price else 'lower' if new_price < original_price else 'same'

# Compare user prediction with actual outcome

if user_prediction == outcome:
    print(f"Correct! The option price is {outcome}.")
    print(f"New Option Price: {new_price:.2f}")
else:
    print(f"Incorrect. The actual outcome is {outcome}.")
    print(f"New Option Price: {new_price:.2f}")


print(f"Answered Outcome: {user_prediction}.")

option type: p
Original Option Price: 5.57
Shocked Variable: sigma
Shock Factor: 1.04
Correct! The option price is higher.
New Option Price: 5.87
Answered Outcome: higher.


In [105]:
import numpy as np
import random
from scipy.stats import norm
from utils.bs_funcs import black_scholes

# Define initial parameters for the main leg
S = random.randint(80, 120)  # Current stock price
K = K + random.randint(-20, 20)  # Strike price
T = random.randint(5, 90)/365    # Time to maturity in years
r = 0.105 # Risk-free interest rate
sigma = random.randint(5, 45) / 100 # Volatility of the underlying stock

# Function to calculate the Greeks
def calculate_greeks(S, K, T, r, sigma, type='c'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    delta = norm.cdf(d1) if type == 'c' else -norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
             - r * K * np.exp(-r * T) * norm.cdf(d2)) if type == 'c' else (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
             + r * K * np.exp(-r * T) * norm.cdf(-d2))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    
    return delta, gamma, theta, vega

# Function to create a new option leg
def create_option_leg(S, K, T, r, sigma, existing_strikes):
    while True:
        new_strike = K + random.randint(-20, 20)
        if new_strike not in existing_strikes:
            break
    option_type = random.choice(['c', 'p'])
    return {
        'S': S,
        'K': new_strike,
        'T': T,
        'r': r,
        'sigma': sigma,
        'type': option_type
    }

# Define the main leg
legs = [create_option_leg(S, K, T, r, sigma, existing_strikes=set([K]))]

# Add additional legs to make a total of 2 legs
num_additional_legs = random.choice([0,1])
existing_strikes = {K}
for _ in range(num_additional_legs):
    leg = create_option_leg(S, K, T, r, sigma, existing_strikes)
    existing_strikes.add(leg['K'])
    legs.append(leg)

# Each leg will be either long or short
sides = [random.choice(['long', 'short']) for _ in range(len(legs))]

# Function to randomly shock one of the variables in a leg
def shock_variable(leg):
    variable_to_shock = random.choice(['S', 'sigma'])
    shock_factor = random.choice([0.9, 1.1])  # Random shock between -10% to +10%
    leg[variable_to_shock] *= shock_factor
    return variable_to_shock, shock_factor, leg

# Calculate the original price and Greeks for the entire structure
def calculate_structure(legs, sides):
    total_price = 0
    total_greeks = np.array([0, 0, 0, 0], dtype=float)
    for leg, side in zip(legs, sides):
        price = black_scholes(**leg)
        if side == 'short':
            price = -price
        total_price += price
        greeks = calculate_greeks(**leg)
        if side == 'short':
            greeks = -np.array(greeks)
        total_greeks += greeks
    return total_price, total_greeks

original_price, original_greeks = calculate_structure(legs, sides)

# Shock one variable in one of the legs
leg_to_shock = random.choice(legs)
variable_to_shock, shock_factor, shocked_leg = shock_variable(leg_to_shock)

# Calculate the new price and Greeks for the entire structure after the shock
new_price, new_greeks = calculate_structure(legs, sides)

# Sort the legs by strike price
legs_sorted_by_strike = sorted(zip(legs, sides), key=lambda x: x[0]['K'])

# Print all options details
print("\nOriginal Option Legs:")
for i, (leg, side) in enumerate(legs_sorted_by_strike):
    print(f"Leg {i + 1}: Type={leg['type']}, Strike={leg['K']}, Position={side}, S={leg['S']}, T={leg['T']:.2f}, r={leg['r']:.2f}, sigma={leg['sigma']:.2f}")

print(f"\nOriginal Structure Price: {original_price:.2f}")
print(f"Original Structure Greeks: Delta={original_greeks[0]:.2f}, Gamma={original_greeks[1]*100:.2f}, Theta={original_greeks[2]:.2f}, Vega={original_greeks[3]:.2f}")
print(f"\nShocked Variable: {variable_to_shock}")
print(f"Shock Factor: {shock_factor:.2f}")

# Ask user for prediction
user_prediction = input("\nPredict the outcome on the structure pricing (higher/lower/same): ").strip().lower()

# Determine actual outcome
outcome = 'higher' if new_price > original_price else 'lower' if new_price < original_price else 'same'

# Compare user prediction with actual outcome
if user_prediction == outcome:
    print(f"\nCorrect! The structure price is {outcome}.")
else:
    print(f"\nIncorrect. The actual outcome is {outcome}.")
print(f"New Structure Price: {new_price:.2f}")
print(f"New Structure Greeks: Delta={new_greeks[0]:.2f}, Gamma={new_greeks[1]*100:.2f}, Theta={new_greeks[2]:.2f}, Vega={new_greeks[3]:.2f}\n")
print(f"Answered Outcome: {user_prediction}.") 



Original Option Legs:
Leg 1: Type=p, Strike=78, Position=long, S=85.5, T=0.16, r=0.10, sigma=0.38

Original Structure Price: 0.45
Original Structure Greeks: Delta=-0.07, Gamma=0.91, Theta=-5.22, Vega=4.88

Shocked Variable: S
Shock Factor: 0.90

Correct! The structure price is higher.
New Structure Price: 1.68
New Structure Greeks: Delta=-0.21, Gamma=2.26, Theta=-9.86, Vega=9.82

Answered Outcome: higher.


## NEW TESTS

In [2]:
import numpy as np
import random
from scipy.stats import norm
from utils.bs_funcs import black_scholes

# Define initial parameters for the main leg
S = random.randint(80, 120)  # Current stock price
K = S + random.randint(1, 5)  # Strike price
T = random.randint(5, 90) / 365  # Time to maturity in years
r = 0.105  # Risk-free interest rate
sigma = random.randint(5, 45) / 100  # Volatility of the underlying stock

# Function to calculate the Greeks
def calculate_greeks(S, K, T, r, sigma, type='c'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    delta = norm.cdf(d1) if type == 'c' else -norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
             - r * K * np.exp(-r * T) * norm.cdf(d2)) if type == 'c' else (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
             + r * K * np.exp(-r * T) * norm.cdf(-d2))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    rho = K * T * np.exp(-r * T) * norm.cdf(d2) if type == 'c' else -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return delta, gamma, theta, vega, rho

# Function to create a new option leg
def create_option_leg(S, K, T, r, sigma, existing_strikes, option_type=None):
    while True:
        new_strike = K + random.randint(-20, 20)
        if new_strike not in existing_strikes:
            break
    if option_type is None:
        option_type = random.choice(['c', 'p'])
    return {
        'S': S,
        'K': new_strike,
        'T': T,
        'r': r,
        'sigma': sigma,
        'type': option_type
    }

# Define the main leg
legs = [create_option_leg(S, K, T, r, sigma, existing_strikes=set([K]))]

# Ensure the second leg is of the same type as the first leg and not a duplicate position
first_leg_type = legs[0]['type']
first_leg_position = random.choice(['long', 'short'])
existing_positions = {(first_leg_type, first_leg_position)}

# Add additional legs to make a total of 2 legs
num_additional_legs = 1
existing_strikes = {K}
for _ in range(num_additional_legs):
    while True:
        leg = create_option_leg(S, K, T, r, sigma, existing_strikes, option_type=first_leg_type)
        position = random.choice(['long', 'short'])
        if (leg['type'], position) not in existing_positions:
            existing_positions.add((leg['type'], position))
            existing_strikes.add(leg['K'])
            legs.append(leg)
            break

# Each leg will be either long or short
sides = list(existing_positions)

# Function to randomly shock one of the variables across all legs
def shock_variable(legs):
    variable_to_shock = random.choice(['S', 'sigma'])
    shock_factor = random.choice([0.9, 1.1])  # Random shock between -10% to +10%
    for leg in legs:
        leg[variable_to_shock] *= shock_factor
    return variable_to_shock, shock_factor, legs

# Calculate the original price and Greeks for the entire structure
def calculate_structure(legs, sides):
    total_price = 0
    total_greeks = np.array([0, 0, 0, 0, 0], dtype=float)  # Including Rho
    for leg, side in zip(legs, sides):
        price = black_scholes(**leg)
        if side[1] == 'short':
            price = -price
        total_price += price
        greeks = calculate_greeks(**leg)
        if side[1] == 'short':
            greeks = -np.array(greeks)
        total_greeks += greeks
    return total_price, total_greeks

original_price, original_greeks = calculate_structure(legs, sides)

# Shock one variable across all legs
variable_to_shock, shock_factor, shocked_legs = shock_variable(legs)

# Calculate the new price and Greeks for the entire structure after the shock
new_price, new_greeks = calculate_structure(shocked_legs, sides)

# Sort the legs by strike price
legs_sorted_by_strike = sorted(zip(legs, sides), key=lambda x: x[0]['K'])

# Print all options details
print("\nOriginal Option Legs:")
for i, (leg, side) in enumerate(legs_sorted_by_strike):
    print(f"Leg {i + 1}: Position={side[1]}, Type={leg['type']}, Strike={leg['K']}, S={leg['S']}, T={leg['T']:.2f}, r={leg['r']:.2f}, sigma={leg['sigma']:.2f}")

print(f"\nOriginal Structure Price: {original_price:.2f}")
print(f"Original Structure Greeks: Delta={original_greeks[0]:.2f}, Gamma={original_greeks[1]*100:.2f}, Theta={original_greeks[2]:.2f}, Vega={original_greeks[3]:.2f}, Rho={original_greeks[4]:.2f}")
print(f"\nShocked Variable: {variable_to_shock}")
print(f"Shock Factor: {shock_factor:.2f}")

# Ask user for prediction
user_prediction = input("\nPredict the outcome on the structure pricing (higher/lower/same): ").strip().lower()

# Determine actual outcome
outcome = 'higher' if new_price > original_price else 'lower' if new_price < original_price else 'same'

# Compare user prediction with actual outcome
if user_prediction == outcome:
    print(f"\nCorrect! The structure price is {outcome}.")
else:
    print(f"\nIncorrect. The actual outcome is {outcome}.")
print(f"New Structure Price: {new_price:.2f}")
print(f"New Structure Greeks: Delta={new_greeks[0]:.2f}, Gamma={new_greeks[1]*100:.2f}, Theta={new_greeks[2]:.2f}, Vega={new_greeks[3]:.2f}, Rho={new_greeks[4]:.2f}\n")
print(f"Answered Outcome: {user_prediction}.")



Original Option Legs:
Leg 1: Position=short, Type=p, Strike=114, S=115.50000000000001, T=0.24, r=0.10, sigma=0.40
Leg 2: Position=long, Type=p, Strike=125, S=115.50000000000001, T=0.24, r=0.10, sigma=0.40

Original Structure Price: 7.87
Original Structure Greeks: Delta=-0.17, Gamma=-0.35, Theta=5.80, Vega=-3.69, Rho=-6.14

Shocked Variable: S
Shock Factor: 1.10

Incorrect. The actual outcome is lower.
New Structure Price: 5.96
New Structure Greeks: Delta=-0.19, Gamma=0.05, Theta=2.39, Vega=0.59, Rho=-6.55

Answered Outcome: higher.
